In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorboard.plugins.hparams import api as hp

In [3]:
def load_data(filepath):
    import h5py
    import numpy as np
    h5f = h5py.File(filepath, 'r')
    X = h5f['X'][:]
    classnames = [s.decode('utf-8') for s in h5f['classname'][:]]
    filenames = [s.decode('utf-8') for s in h5f['filename'][:]]
    h5f.close()
    return X, np.array(classnames), np.array(filenames)

def create_train_test_idx(classnames):
    import pandas as pd
    df = pd.DataFrame(data=enumerate(classnames), columns=['index', 'classname'])
    test_df = None
    train_df = None
    for classname in df['classname'].unique():
        test_tmp_df = df[df['classname']==classname].sample(50, replace=False, random_state=1234)
        train_tmp_df = df[(df['classname']==classname) & ~(df['index'].isin(test_tmp_df['index']))]
        test_df = test_tmp_df if test_df is None else pd.concat([test_df, test_tmp_df])
        train_df = train_tmp_df if train_df is None else pd.concat([train_df, train_tmp_df])    
    return train_df['index'].values, test_df['index'].values
    
X, classnames, filenames = load_data('data/RockAI_images_224x224.h5')
train_idx, test_idx = create_train_test_idx(classnames)
X_train, X_test = X[train_idx], X[test_idx]
y_train = [classnames[i] for i in train_idx]
y_test = [classnames[i] for i in test_idx]

num_classes = 2

#Pre-process the data
X_train = tf.keras.applications.vgg16.preprocess_input(X_train)
X_test = tf.keras.applications.vgg16.preprocess_input(X_test)
y_train = [0 if x=='No_RA' else 1 for x in y_train]
y_test = [0 if x=='No_RA' else 1 for x in y_test]
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [4]:
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#    shear_range=0.2,
#     zoom_range=0.2,
#     rotation_range = 30,
#     horizontal_flip=True,
#     vertical_flip=True)
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
datagen.fit(X_train)

In [5]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([256, 512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.25, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC, display_name=METRIC)],
    )


def train_test_model(hparams):
    vgg_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in vgg_model.layers[:]:
        layer.trainable = False
    # for i, layer in enumerate(vgg_model.layers):
    #     print(i, layer.name, layer.trainable)
        
    model = tf.keras.models.Sequential([
        vgg_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(2, activation=tf.nn.softmax),
    ])
    model.compile(
        loss='categorical_crossentropy',
        optimizer=hparams[HP_OPTIMIZER],        
        metrics=[METRIC],
    )

    model.fit(X_train, y_train, epochs=30, verbose=1)
    _, score = model.evaluate(X_test, y_test)
    return score


def run(run_dir, hparams, log_result_filepath):
    import datetime
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        score = train_test_model(hparams)
        tf.summary.scalar(METRIC, score, step=1)
    with open(log_result_filepath, 'a') as fp:
        fp.write(f"timestamp: {datetime.datetime.now()}, ")
        for h in hparams:
            fp.write(f"{h.name}: {hparams[h]}, ")
        fp.write("data_augmentation: no, epochs: 30 --> ")
        fp.write(f"{METRIC}:{score}\n")

In [6]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            for _ in range(3):
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams, 'logs/hparam_tuning_score.txt')
                session_num += 1

--- Starting trial: run-0
{'num_units': 256, 'dropout': 0.25, 'optimizer': 'adam'}
Epoch 1/30
43/43 [==============================] - 9s 103ms/step - loss: 3.7020 - accuracy: 0.9125
Epoch 2/30
43/43 [==============================] - 3s 68ms/step - loss: 0.0708 - accuracy: 0.9824
Epoch 3/30
43/43 [==============================] - 3s 69ms/step - loss: 0.0256 - accuracy: 0.9934
Epoch 4/30
43/43 [==============================] - 3s 68ms/step - loss: 0.0061 - accuracy: 0.9986
Epoch 5/30
43/43 [==============================] - 3s 69ms/step - loss: 0.0018 - accuracy: 0.9999
Epoch 6/30
43/43 [==============================] - 3s 69ms/step - loss: 0.0185 - accuracy: 0.9979
Epoch 7/30
43/43 [==============================] - 3s 69ms/step - loss: 2.4852e-05 - accuracy: 1.0000
Epoch 8/30
43/43 [==============================] - 3s 69ms/step - loss: 0.0142 - accuracy: 0.9982
Epoch 9/30
43/43 [==============================] - 3s 69ms/step - loss: 0.0107 - accuracy: 0.9982
Epoch 10/30
43/43 [==